In [23]:
from IPython.display import display
import dash
import sys,os
if  not os.path.abspath('./') in sys.path:
    sys.path.append(os.path.abspath('./'))
if  not os.path.abspath('../') in sys.path:
    sys.path.append(os.path.abspath('../'))
sys.path.append(os.path.abspath('../../dashgrid'))
sys.path.append(os.path.abspath('../../dashgrid/dashgrid'))
from dashgrid import dgrid
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output,State
import dash_table
import pandas as pd
import numpy as np
from dashgrid import dgrid
import importlib
# importlib.reload(dgrid)


#### Create styles to be used in grids

In [24]:
button_style={
    'line-height': '40px',
    'borderWidth': '1px',
    'borderStyle': 'dashed',
    'borderRadius': '1px',
    'textAlign': 'center',
    'background-color':'#fffff0',
    'vertical-align':'middle',
}
button_style_no_border={
    'line-height': '40px',
#     'borderWidth': '1px',
#     'borderStyle': 'dashed',
#     'borderRadius': '1px',
    'textAlign': 'center',
    'background-color':'#fffff0',
    'vertical-align':'middle',
}

#### Create a test grid, with grids inside of grids

In [25]:
def dash_app_1():
    d1 = dgrid.GridItem(html.Div('test 1',style=button_style))
    d2 = dgrid.GridItem(html.Div('test 2',style=button_style))
    g1 = dgrid.create_grid([d1,d2],additional_grid_properties_dict={'border':'none'})
    d3 = dgrid.GridItem(html.Div('test 3',style=button_style))
    g2 = dgrid.create_grid([g1,d3])
    app = dash.Dash()
    app.layout = html.Div(g2)
    return app
# app.run_server(host='127.0.0.1',port=8500)

#### Create a method that creates dash_table's

In [26]:
def create_dt_div(dtable_id,df_in=None,
                  columns_to_display=None,
                  editable_columns_in=None,
                  title='Dash Table'):
    editable_columns = [] if editable_columns_in is None else editable_columns_in
    datatable_id = dtable_id
    dt = dash_table.DataTable(
        page_current= 0,
        page_size= 100,
        page_action='native',
        sort_action='native',
        filter_action='none', # 'fe',
#             content_style='grow',
        style_cell_conditional=[
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(248, 248, 248)'
            }
        ] + [
            {
                'if': {'column_id': c},
                'textAlign': 'left',
            } for c in ['symbol', 'underlying']
        ],

        style_as_list_view=False,
        style_table={
            'maxHeight':'450','overflowX': 'scroll',
        } ,
        editable=True,
        css=[{"selector": "table", "rule": "width: 100%;"}],
        id=datatable_id
    )
    if df_in is None:
        df = pd.DataFrame({'no_data':[]})
    else:
        df = df_in.copy()
        if columns_to_display is not None:
            df = df[columns_to_display]                
    dt.data=df.to_dict("rows")
    dt.columns=[{"name": i, "id": i,'editable': True if i in editable_columns else False} for i in df.columns.values]                    
    print(f'dt.columns {dt.columns}')
    child_div = html.Div([html.H4(title,style={'height':'3px'}),dt])
    return child_div

def file_store_transformer(contents):
    if contents is None or len(contents)<=0 or contents[0] is None:
        d =  None
    else:
        d = dgrid.parse_contents(contents).to_dict('rows')
    return d


#### Create a dash app - WITHOUT GRIDS - that displays an Upload button, a dash_table and a dash Graph

In [52]:
def dash_app_2():
    u1 = dcc.Upload(
                id='u1',
                children=html.Div(['enter a csv file']),
                # Allow multiple files to be uploaded
                multiple=False,
                style=button_style)

    df_init = pd.DataFrame({'symbol':['spy','ibm'],'position':[100,-100]})
    cols=['symbol','position']
    s1 = dcc.Store(id='s1',data=df_init.to_dict('rows'))
    s2 = dcc.Store(id='s2')

    dtable_div = create_dt_div('dtable',df_in=df_init,
                               columns_to_display=cols,
                               editable_columns_in=['position'])
    dt1 = html.Div([dtable_div],id='dt1')
    gr = dgrid.GridGraph('g1', 'graph 1',(dt1.id,'data'),
                        df_x_column='symbol',df_y_columns=['position'],plot_bars=True)
    gr1 = gr.html
    gr1.id = 'gr1'

    u1_lambda = lambda value_list: [None,None] if value_list[0] is None else [file_store_transformer(value_list[0]),None]
    u1_comp = dgrid.ComponentWrapper(u1,
                     input__tuples=[(u1.id,'contents')],
                     output_tuples=[(s1.id,'data'),(s2.id,'data')],
                     callback_input_transformer=u1_lambda)

    # dt1_lambda = lambda value_list: [None] if value_list[0] is None else [create_dt_div('dtable',df_in=pd.DataFrame(value_list[0]),editable_columns_in=['position]'])]
    def dt1_lambda(value_list): 
        print(f'dt1_lambda value_list: {value_list}')
        ret = [None,None]
        try:
            if value_list[0] is not None:
                dict_df = value_list[0]
                df = pd.DataFrame(dict_df)
                dt_div = create_dt_div('dtable',df_in=df,
                            columns_to_display=cols,
                            editable_columns_in=['position'])
                ret =  [dt_div,dict_df]
        except Exception as e:
            print(str(e))
        print(f'dt1_lambda ret: {ret}')
        if ret[0] is None:
            raise ValueError('dt1_lambda value_list has no data.  Callback return is ignored')
        return ret

    dt1_comp = dgrid.ComponentWrapper(dt1,
                     input__tuples=[(s1.id,'data')],
                     output_tuples=[(dt1.id,'children'),('dtable','data')],
                     callback_input_transformer=dt1_lambda)


    def gr1_lambda(value_list): 
        print(f'gr1_lambda value_list: {value_list}')
        ret = [None]
        try:
            if value_list[0] is not None:
                df = pd.DataFrame(value_list[0])
                ret =  [gr.make_chart_figure(df)]
        except Exception as e:
            print(str(e))
        print(f'gr1_lambda ret: {ret}')
        return ret

    gr1_comp = dgrid.ComponentWrapper(gr1,
                     input__tuples=[('dtable','data')],
                     output_tuples=[(gr1.children.id,'figure')],
                     callback_input_transformer=gr1_lambda)

    app = dash.Dash()
    app.layout = html.Div([u1_comp.html,s1,s2,dt1_comp.html,gr1_comp.html])
    # [c.callback(app) for c in [u1_comp,dt1_comp]]
    u1_comp.callback(app)
    dt1_comp.callback(app)
    gr1_comp.callback(app)
    return app


#### Create app above, but with Grid wrappers

In [87]:
def dash_app_3():
    app = dash_app_2()    
    apc = app.layout.children
    upl = apc[0]
    stores = [apc[1],apc[2]]
    gr = dgrid.create_grid([apc[3],apc[4]])
    app.layout = html.Div([upl,gr]+stores)
    return app


In [96]:
class UploadComponent(dgrid.ComponentWrapper):
    def __init__(self,component_id,text=None,style=None,logger=None):
        t = "Choose a File" if text is None else text
        u1 = dcc.Upload(
                    id=component_id,
                    children=html.Div([t]),
                    # Allow multiple files to be uploaded
                    multiple=False,
                    style=button_style if style is None else style)
        
        u1_lambda = lambda value_list: [None] if value_list[0] is None else [file_store_transformer(value_list[0])]
        self.s1 = dcc.Store(id=component_id+"_store")
        super(UploadComponent,self).__init__(u1,
                     input__tuples=[(u1.id,'contents')],
                     output_tuples=[(self.s1.id,'data')],
                     callback_input_transformer=u1_lambda,logger=logger)
        
    @dgrid.ComponentWrapper.html.getter
    def html(self):
        return html.Div([self.div,self.s1])
        
class DashTableComponent(dgrid.ComponentWrapper):
    def __init__(self,component_id,df_initial,input_tuples,title=None,
                 editable_columns=None,style=None,logger=None):
        self.component_id = component_id
        self.html_id = component_id+'_html'
        
        # create initial div
        dtable_div = create_dt_div(self.component_id,df_in=df_initial,
                        columns_to_display=df_initial.columns,
                        editable_columns_in=editable_columns,
                        title='Dash Table' if title is None else title)
        outer_div = html.Div([dtable_div],id=self.html_id)
        
        # define dash_table callback using closure so that you don't refer back to 
        #   class instance during the callback
        def _create_dt_lambda(component_id,cols,editable_cols,logger):
            def _dt_lambda(value_list): 
                logger.debug(f'dt_lambda value_list: {value_list}')
                ret = [None,None]
                try:
                    if value_list[0] is not None:
                        dict_df = value_list[0]
                        df = pd.DataFrame(dict_df)
                        dt_div = create_dt_div(component_id,df_in=df,
                                    columns_to_display=cols,
                                    editable_columns_in=editable_cols)
                        ret =  [dt_div,dict_df]
                except Exception as e:
                    logger.warn(str(e))
                logger.debug(f'dt1_lambda ret: {ret}')
                if ret[0] is None:
                    raise ValueError('dt1_lambda value_list has no data.  Callback return is ignored')
                return ret
            return _dt_lambda
        
        
        # do super, but WITHOUT the callback
        super(DashTableComponent,self).__init__(outer_div,
                     input__tuples=input_tuples,
                     output_tuples=[
                         (outer_div.id,'children'),
                         (self.component_id,'data')],
                     callback_input_transformer=lambda v:None,logger=logger)

        # define callback so that it includes self.logger
        dtlam = _create_dt_lambda(self.component_id,df_initial.columns.values,editable_columns,self.logger)
        self.callback_input_transformer = dtlam
        

In [101]:
def dash_app_4():
    u1_comp = UploadComponent('u1','choose a csv portfolio file with symbol and position columns')
    df_init = pd.DataFrame({'symbol':['spy','ibm'],'position':[100,-100]})

    dt1_comp = DashTableComponent('dt1',df_init,u1.output_tuples,editable_columns=['position'])
    app = dash.Dash()
    app.layout = html.Div([u1_comp.html,dt1_comp.html])#,gr1_comp.html])
    # [c.callback(app) for c in [u1_comp,dt1_comp]]
    u1_comp.callback(app)
    dt1_comp.callback(app)
    return app




In [102]:
DASH_APP_TO_RUN = dash_app_4

In [ ]:
app = DASH_APP_TO_RUN()
app.run_server(host='127.0.0.1',port=8500)

dt.columns [{'name': 'position', 'id': 'position', 'editable': True}, {'name': 'symbol', 'id': 'symbol', 'editable': False}]
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


2019-07-19 13:14:28,438 - werkzeug - INFO -  * Running on http://127.0.0.1:8500/ (Press CTRL+C to quit)
2019-07-19 13:14:32,536 - werkzeug - INFO - 127.0.0.1 - - [19/Jul/2019 13:14:32] "GET / HTTP/1.1" 200 -
2019-07-19 13:14:32,581 - werkzeug - INFO - 127.0.0.1 - - [19/Jul/2019 13:14:32] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.0.0&m=1563025507 HTTP/1.1" 200 -
2019-07-19 13:14:32,604 - werkzeug - INFO - 127.0.0.1 - - [19/Jul/2019 13:14:32] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.0.0&m=1563025507 HTTP/1.1" 200 -
2019-07-19 13:14:32,617 - werkzeug - INFO - 127.0.0.1 - - [19/Jul/2019 13:14:32] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.0.0&m=1563025507 HTTP/1.1" 200 -
2019-07-19 13:14:32,635 - werkzeug - INFO - 127.0.0.1 - - [19/Jul/2019 13:14:32] "GET /_dash-component-suites/dash_core_components/highlight.pack.js?v=1.0.0&m=1563025508 HTTP/1.1" 200 -
2019-07-19 13:14:32,658 - werkzeug - INFO - 127.0.0.1 

dt.columns [{'name': 'position', 'id': 'position', 'editable': True}, {'name': 'symbol', 'id': 'symbol', 'editable': False}]


In [31]:
for c in app.layout.children:
    display(c)
    

Div([Upload(children=Div(['enter a csv file']), id='u1', multiple=False, style={'line-height': '40px', 'borderWidth': '1px', 'borderStyle': 'dashed', 'borderRadius': '1px', 'textAlign': 'center', 'background-color': '#fffff0', 'vertical-align': 'middle'})])

Store(id='s1', data=[{'position': 100, 'symbol': 'spy'}, {'position': -100, 'symbol': 'ibm'}])

Store(id='s2')

Div([Div(children=[Div([H4(children='Dash Table', style={'height': '3px'}), DataTable(columns=[{'name': 'symbol', 'id': 'symbol', 'editable': False}, {'name': 'position', 'id': 'position', 'editable': True}], css=[{'selector': 'table', 'rule': 'width: 100%;'}], data=[{'symbol': 'spy', 'position': 100}, {'symbol': 'ibm', 'position': -100}], editable=True, id='dtable', style_as_list_view=False, page_action='native', page_current=0, page_size=100, filter_action='none', sort_action='native', style_table={'maxHeight': '450', 'overflowX': 'scroll'}, style_cell_conditional=[{'if': {'row_index': 'odd'}, 'backgroundColor': 'rgb(248, 248, 248)'}, {'if': {'column_id': 'symbol'}, 'textAlign': 'left'}, {'if': {'column_id': 'underlying'}, 'textAlign': 'left'}])])], id='dt1')])

Div([Div(children=Graph(id='g1', figure=Figure({
    'data': [{'type': 'bar', 'uid': '95e78a12-aa1e-11e9-b585-8c859025e7f9', 'x': [], 'y': []}],
    'layout': {'plot_bgcolor': '#f5f5f0', 'title': 'graph 1'}
})), id='gr1', className='item1', style={'margin-right': 'auto', 'margin-left': 'auto', 'height': '98%', 'width': '98%', 'border': 'thin solid'})])

In [79]:
importlib.reload(dgrid)

<module 'dashgrid.dgrid' from '/Users/bperlman1/Documents/billybyte/pyliverisk/dashgrid/dashgrid/dgrid.py'>